In [1]:
import nest_asyncio     # needed in order to get
nest_asyncio.apply()    # pystan to run properly
###
import stan
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from pathlib import Path
from patsy import dmatrix, dmatrices

import theano


import aesara_theano_fallback.tensor as tt

RANDOM_SEED = 1

NUM_CHAINS = 2
NUM_SAMPLES = 1000

# https://linuxtut.com/en/c17070e3b242b0d62c39/
def waic(fit):
    sel_col = [col for col in fit.to_frame() if col.startswith('log_lik')]
    log_lik = fit.to_frame()[sel_col]
    lppd = np.log(np.exp(log_lik).mean(axis=0)).sum()
    p_waic = np.var(log_lik, axis=0).sum()
    waic = -2*lppd + 2*p_waic
    return round(waic, 3)


fn = 'data.csv'

data_fp = Path(fn)
data = pd.read_csv(data_fp)
data = data.rename(columns={'zscores': 'zscore'})
data = data[data['island'] != 'filler']
data = data.reset_index(drop=True)

data['subject'] = data['subject'].astype('category')
data['judgment'] = (data['judgment']-1).astype('category')
data['item'] = data['item'].astype('category')
data['dependency'] = data['dependency'].astype('category')
data['island'] = data['island'].astype('category')
data['structure'] = data['structure'].astype('category')
data['distance'] = data['distance'].astype('category')

data['itemID'] = data['item'].cat.codes
data['itemID'] = data['itemID'].astype('category')
data['subjectID'] = data['subject'].cat.codes
data['subjectID'] = data['subjectID'].astype('category')
data['islandID'] = data['island'].cat.codes
data['islandID'] = data['islandID'].astype('category')

model_code = """
data {
    int<lower=1> N; // number of samples/observations
    int<lower=0> K; // number of predictors - 1 (intercept)
    matrix[N,K]  X; // matrix of predictors
    vector[N]    y; // observed zscores
}
parameters {
    real alpha;
    vector[K] beta;
    real<lower=0> sigma;
}
model {
    // priors
    // likelihood
    y ~ normal(X * beta, sigma); 
}
generated quantities {
    vector[N] log_lik;
    for (i in 1:N)
        log_lik[i] = normal_lpdf(y[i] | X * beta, sigma);
}
"""
X = dmatrices("zscore ~ dependency*island*structure*distance", data)
y = [x[0] for x in X[0]]
model_data = {
    'N': X[1].shape[0],
    'K': X[1].shape[1],
    'X': X[1],
    'y': y,
}

In [ ]:
posterior = stan.build(model_code, model_data, 
                       random_seed=RANDOM_SEED)

In [ ]:
# num chains & samples set low just to make sure it runs
fit = posterior.sample(num_chains=4,
                       num_samples=1000 
)
fit_df = fit.to_frame()

In [ ]:
waic(fit)